In [ ]:
import os
import sys
import django
import pandas as pd

from IPython.display import display
from pathlib import Path

CWD = Path(".").resolve()
DBF_APP_PATH = Path(CWD.parent.parent, "dbfapp")

#It is necessary to run django setup to know desulf package from app
#Is it necessary to add the path to the app to the environment
#variable path, to it possible run django know app module dbfapp.
sys.path += [str(DBF_APP_PATH)]

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dbfapp.settings")
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "True")
django.setup()

from modelbackend.management.commands.runmodels import calculate_model
from dbfcore.settings import TZ_UTC
from dbfcore.dataset.hooks import get_datasources_configured_with_env
from dbfcore.predictionmodel import DummyBlastFurnaceModel

[2024-10-07 13:30:56,830] INFO  54174 140235365660480 datasets     PyTorch version 2.1.2 available.


In [2]:
# os.environ["PI_CLIENT_USER"] = "DIGITAL_DBF"
# os.environ["PI_CLIENT_PASSWORD"] = "N3v3R#3xPiR3"
# os.environ["PI_API_ENDPOINT"] = "https://vpiaf/piwebapi"
# os.environ["PI_DB_NAME"] = "VPIPRIMSK"

In [5]:
bf_model = DummyBlastFurnaceModel(1, "TEMPERATURE")
calc_time = pd.Timestamp("2024-06-01 13:00:00", tzinfo=TZ_UTC)
datasources = get_datasources_configured_with_env()
result = calculate_model(bf_model, calc_time, datasources)

Loading bf1_charge_flows
Loading bf1_tappings


In [35]:
result.data

{Timestamp('2024-06-01 13:00:00+0000', tz='UTC'): 10}

In [16]:
(pd.Timedelta("0 days 12.5 seconds") / 2).round("s")

Timedelta('0 days 00:00:06')

In [ ]:
import torch
torch.set_grad_enabled(False)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from modelbackend.management.commands.runmodels import get_autoencoder, float32_bytes_to_flat_tensor
from dbfcore.liveprediction.livepidatasource import live_pi_data_loader

In [ ]:
from modelbackend.models import ModelInputOutput, PredictionModel, PredictionResult

In [ ]:
model = PredictionModel.objects.all().first()
model

In [ ]:
outputs = list(ModelInputOutput.objects.filter(is_input=False, model=model))
outputs

In [ ]:
encoder = get_autoencoder(outputs[0].signalencoder.encoder)

In [ ]:
n = 50
last_n_predictions = PredictionResult.objects.filter(model=model).order_by("-prediction_start")[:n]

In [ ]:
def get_prediction_df(prediction_result, decoder):
    test_times = torch.arange(0, 3600 + 60, 60)
    raw_decoded = decoder.decode(float32_bytes_to_flat_tensor(prediction_result.encoded)[:10].unsqueeze(0), test_times.unsqueeze(0))[0]
    times_df = pd.DataFrame(test_times, columns=["time"]).astype("timedelta64[s]") + prediction_result.prediction_start
    predictions_df = pd.DataFrame(raw_decoded, columns=["value"])
    prediction_time_df = pd.DataFrame([prediction_result.prediction_start] * len(times_df), columns=["prediction_time"])
    return pd.concat([prediction_time_df, times_df, predictions_df], axis=1)

In [ ]:
predictions = pd.concat([get_prediction_df(result, encoder) for result in last_n_predictions])
measured_temps = live_pi_data_loader("SK1.HotMetal.Temp.MeasDevice.C", predictions["time"].max() - pd.Timedelta("6h"), predictions["time"].max())

In [ ]:
fig = px.line(predictions, x="time", y="value", color="prediction_time")
fig.add_trace(go.Scatter(name="real", x=measured_temps.index, y=measured_temps.value, mode="markers", marker={"size": 15, "symbol":"x"}))